# Import and Set

In [3]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import numpy as np
import os
import pickle
from torchaudio.models import Conformer
import math
from torch import nn, Tensor
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import Normalizer, StandardScaler
from einops.layers.torch import Rearrange, Reduce
from utils import *
from model import VT_CNN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_class = 1
stock_symbol = '5871.TW'
end_date = '2024-12-31'

init = True
fp16_training = False
num_epochs = 500
config = {
    'lr': 0.001,
}

# Init

In [4]:
with open('../DataLoader/dataloader.pk', 'rb') as f:
    data = pickle.load(f)
dataloader_train = data['trainloader']
dataloader_valid = data['validloader']
# dataloader_test = data['testloader']

# Define model
### Question
- Conformer include decoder?

In [5]:
# Vision Transformer: https://github.com/pytorch/vision/blob/main/torchvision/models/vision_transformer.py
# try pretrain, same link above
class VT_CNN(nn.Module):
    def __init__(self, num_class=2):
        super(VT_CNN, self).__init__()

        # =======
        # Unet
        self.conv_init = nn.Conv2d(5, 3, kernel_size=3, stride=1, padding=1, bias=False)
        self.VisionTransformer = torchvision.models.VisionTransformer(
            image_size = 100,
            patch_size = 10,
            num_layers = 16,
            num_heads = 5,
            hidden_dim = 250,
            mlp_dim = 520
        )
        self.ln_init = nn.LayerNorm(1000)
        self.fc = nn.Linear(1000, num_class)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Input scale: (0, 255)
        Output scale: (0, 255)
        """
        
        x = self.conv_init(x)
        x = self.VisionTransformer(x)
        x = self.ln_init(x)
        x = self.relu(x)        
        x = self.fc(x)

        return x
    

In [6]:
"""
x = 123
def global_var():
    global x
    x = "awesome"
    print(x)
print(x)
global_var()
"""

'\nx = 123\ndef global_var():\n    global x\n    x = "awesome"\n    print(x)\nprint(x)\nglobal_var()\n'

# Train

In [7]:
"""
Choose if fp16 and define model
"""
# !pip install accelerate==0.2.0
# Model
if fp16_training:
    print('Accelerating')
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device
    model = VT_CNN()
else:
    model = VT_CNN().to(device)

"""
Init for models, learning rate, ...
"""
if os.path.exists(f'Temp//Conformer_{stock_symbol}_LastTrainInfo.pk'):
    if init:
        print("Init model")
        lr = config['lr']
        last_epoch = 0
        min_val_loss = 10000
        loss_train = []
        loss_valid = []
    else:
        print('Load from last train epoch')
        with open(f'Temp//Conformer_{stock_symbol}_LastTrainInfo.pk', 'rb') as f:
            last_train_info = pickle.load(f)
        lr = last_train_info['lr']
        last_epoch = last_train_info['epoch']
        min_val_loss = last_train_info['min val loss']
        model.load_state_dict(torch.load(f'Temp//Conformer_{stock_symbol}_checkpoint_LastTrainModel.pt'))
        with open(f'Temp//Conformer_{stock_symbol}_TrainValHistLoss.pk', 'rb') as f:
            loss_train_val = pickle.load(f)
        loss_train = loss_train_val['train']
        loss_valid = loss_train_val['valid']
else:
    print("Init model")
    lr = config['lr']
    last_epoch = 0
    min_val_loss = 10000.0
    loss_train = []
    loss_valid = []
print(f'Last train epoch: {last_epoch}  '
        f'Last train lr: {lr}   '
        f'Min val loss: {min_val_loss}')

import torch.optim as optim
import pickle

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=len(dataloader_train)*10, gamma=0.9)        

# Prepare
if fp16_training:
    print('Accelerate Prepare')    
    model, optimizer, dataloader_train, dataloader_valid, scheduler = \
        accelerator.prepare(model, optimizer, dataloader_train, dataloader_valid, scheduler)

Init model
Last train epoch: 0  Last train lr: 0.001   Min val loss: 10000


In [8]:
for epoch in range(last_epoch, num_epochs):
    # Training phase
    model.train()
    loss_train_e = 0
    for batch_x, batch_y in tqdm(dataloader_train):
        # batch_x = mask(batch_x)
        if not fp16_training:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)

        # Loss
        loss = criterion(outputs, batch_y*100)
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        optimizer.step()
        scheduler.step()
        loss_train_e += loss.item()
        
    loss_train_e /= len(dataloader_train)
    loss_train.append(loss_train_e)
    
    loss_valid_e = 0
    with torch.no_grad():
        model.eval()
        for batch_x_val, batch_y_val in tqdm(dataloader_valid):
            # batch_x_val = mask(batch_x_val)
            if not fp16_training:
                batch_x_val = batch_x_val.to(device)
                batch_y_val = batch_y_val.to(device)
            outputs_val = model(batch_x_val)
            loss = criterion(outputs_val, batch_y_val)
            loss_valid_e += loss.item()
        loss_valid_e /= len(dataloader_valid)
        loss_valid.append(loss_valid_e)
            
        torch.save(model.state_dict(), f'Temp/Conformer_{stock_symbol}_checkpoint_LastTrainModel.pt')
        if loss_valid_e < min_val_loss:
            min_val_loss = loss_valid_e
            print(f'New best model found in epoch {epoch} with val loss: {min_val_loss}')
            torch.save(model.state_dict(), f'ConformerResult/Conformer_{stock_symbol}_best_model.pt')            
        if epoch % 50 == 0:
            pass
            # torch.save(model, f'ConformerResult/Conformerr_{stock_symbol}_checkpoint_{epoch}.pt')
            
    with open(f'Temp/Conformer_{stock_symbol}_TrainValHistLoss.pk', 'wb') as f:
        pickle.dump({'train': loss_train, 'valid': loss_valid}, f)
    with open(f'Temp/Conformer_{stock_symbol}_LastTrainInfo.pk', 'wb') as f:
        pickle.dump({'min val loss': min_val_loss, 'epoch': epoch, 'lr': optimizer.param_groups[0]['lr']}, f)
        
    # Print statistics
    print(f'Epoch [{epoch}/{num_epochs}]',
        f'Training Loss: {loss_train_e:.10f}',
        f'Valid Loss: {loss_valid_e:.10f}')

  0%|          | 0/65 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

New best model found in epoch 0 with val loss: 0.0013352816801189499
Epoch [0/500] Training Loss: 3.4953858183 Valid Loss: 0.0013352817


  2%|▏         | 1/65 [00:00<00:07,  8.00it/s]

Epoch [1/500] Training Loss: 3.4907736540 Valid Loss: 0.0023617290


...

Epoch [11/500] Training Loss: 3.4805311075 Valid Loss: 0.0140351903


  2%|▏         | 1/65 [00:00<00:07,  8.02it/s]

Epoch [12/500] Training Loss: 3.4804349936 Valid Loss: 0.0146299973


 65%|██████▍   | 42/65 [00:05<00:03,  7.48it/s]


KeyboardInterrupt: 

In [ ]:
outputs.shape, batch_y.shape

(torch.Size([32, 1]), torch.Size([32, 1]))

# Validate Model

In [ ]:
def load_model():
    import torch
    model = torch.load(f'ConformerResult/Conformer_{stock_symbol}_best_model.pt')
    return model
model = load_model()

In [ ]:

import gc
def test():
    dataloader = dataloader_test

    model.eval()
    s_pred = []
    s_true = []
    for x, y in tqdm(dataloader):
        y_pred = model(x)
        s_pred.append(y_pred.detach())
        s_true.append(y)
    y_pred_tensor = torch.concat(s_pred)
    y_test_tensor = torch.concat(s_true)
    accuracy = (torch.sign(y_pred_tensor) == torch.sign(y_test_tensor)).sum() / len(y_test_tensor)
    return y_pred_tensor, accuracy

y_pred, acc = test()
print(acc)

In [ ]:
# Derive y_pred and y_train_pred of shape(N, 2) and numpy type

y_pred_numpy = y_pred.cpu().numpy()

# predict with train set
y_train_pred = model(torch.tensor(X[-100:], dtype = torch.float32))
y_train_numpy = y_train_pred.detach().cpu().numpy()


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Scaling
prediction = pd.DataFrame(y_pred_numpy)
scaler = StandardScaler()
scaler.fit(y_train_numpy)
prediction = pd.DataFrame(scaler.transform(prediction))

# Get the predicted price of O and C and Prediction merge with complete data
prediction.columns = ['pred_do_1', 'pred_dc_1']
prediction['Date'] = date

true_and_pred = pd.merge(df.reset_index(), prediction, on = 'Date', how = 'left')
true_and_pred['pred_o'] = (true_and_pred['Open'] * (1 + true_and_pred['pred_do_1'])).shift(1)
true_and_pred['pred_c'] = (true_and_pred['Close'] * (1 + true_and_pred['pred_dc_1'])).shift(1)
true_and_pred['pred_oc'] = true_and_pred['pred_c'] - true_and_pred['pred_o']
true_and_pred['true_oc'] = true_and_pred['Close'] - true_and_pred['Open']

# Backtest
asset_list = []
df_backtest = true_and_pred[['Open', 'Close', 'true_oc', 'pred_oc']].dropna()
asset = 1
for index, (o, c, true, pred) in df_backtest.iterrows():
    if pred > 0:
        returns = true/o
        asset *= (1 + returns)
    asset_list.append(asset)

print(asset)
plt.plot(asset_list, label = 'resnet')
plt.plot(df_backtest.reset_index()['Close']/df_backtest['Close'].iloc[0], label = 'buy hold')
plt.legend()
plt.savefig('/ConformerResult/test_backtest.jpg')
# plt.show()